In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import math
from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
%matplotlib inline

In [85]:
df = pd.read_csv("newyork_crime_2013.csv").fillna(0)
df

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
0,Adams Village,"1,861",0,0,0,0,0,12,2,10,0,0.0
1,Addison Town and Village,"2,577",3,0,0,0,3,24,3,20,1,0.0
2,Akron Village,"2,846",3,0,0,0,3,16,1,15,0,0.0
3,Albany,"97,956",791,8,30,227,526,"4,090",705,"3,243",142,0.0
4,Albion Village,"6,388",23,0,3,4,16,223,53,165,5,0.0
5,Alfred Village,"4,089",5,0,0,3,2,46,10,36,0,0.0
6,Allegany Village,"1,781",3,0,0,0,3,10,0,10,0,0.0
7,Amherst Town,"118,296",107,1,7,31,68,"2,118",204,"1,882",32,3.0
8,Amityville Village,"9,519",9,0,2,4,3,210,16,188,6,1.0
9,Amsterdam,"18,182",30,0,0,12,18,405,99,291,15,0.0


In [86]:
#convert from str to int
df["Population"] = df["Population"].str.replace(",", "")

df["PopulationSq"] = np.square(pd.to_numeric(df["Population"]))
#df["PopulationSq"] = float(df["Population"]) ** 2
df["Aggravated\nassault"] = pd.to_numeric(df["Aggravated\nassault"].str.replace(",", ""))
df["Burglary"] = pd.to_numeric(df["Burglary"].str.replace(",", ""))
df["Larceny-\ntheft"] = pd.to_numeric(df["Larceny-\ntheft"].str.replace(",", ""))
df["Motor\nvehicle\ntheft"] = pd.to_numeric(df["Motor\nvehicle\ntheft"].str.replace(",", ""))
df["Robbery"] = pd.to_numeric(df["Robbery"].str.replace(",", ""))

In [87]:
df["Murder"] = np.where(df['Murder and\nnonnegligent\nmanslaughter'] > 0, 1,0)

In [88]:
list(df)

['City',
 'Population',
 'Violent\ncrime',
 'Murder and\nnonnegligent\nmanslaughter',
 'Rape\n(legacy\ndefinition)2',
 'Robbery',
 'Aggravated\nassault',
 'Property\ncrime',
 'Burglary',
 'Larceny-\ntheft',
 'Motor\nvehicle\ntheft',
 'Arson',
 'PopulationSq',
 'Murder']

In [89]:
df["Property\ncrime"] = df["Property\ncrime"].str.replace(",", "")
df["Property\ncrime"] = pd.to_numeric(df["Property\ncrime"])

In [90]:
df["RobberyOH"] = np.where(pd.to_numeric(df["Robbery"].str.replace(",", "")) > 0, 1, 0)

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [117]:
#first let's try vanilla logistic regression

from sklearn import linear_model
from sklearn.model_selection import cross_val_score

regr = linear_model.LogisticRegression(penalty='l1')
Y = df["Murder"].values.reshape(-1, 1)
#
X = df[["Population", "PopulationSq","Arson","Robbery", "Motor\nvehicle\ntheft", "Burglary", "Larceny-\ntheft"]]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=.3)

print(regr.fit(X_train, y_train).score(X_test, y_test))

print(regr.fit(X, Y).score(X, Y))
cross_val_score(regr, X, Y, cv=10)

c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Future

0.895238095238


c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.89367816092


c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWa

array([ 0.94285714,  0.8       ,  0.88571429,  0.8       ,  0.88571429,
        0.91428571,  0.85714286,  0.88571429,  0.97142857,  0.90909091])

In [128]:
#let's try ridge regression

ridgeregr = linear_model.Ridge(alpha=.5, fit_intercept=False, normalize=True) 
ridgeregr.fit(X_train, y_train)
print(ridgeregr.score(X_test, y_test))
#origparams = ridgeregr.coef_[0]
#print(origparams)
cross_val_score(ridgeregr, X, Y, cv=10)

0.309119759346


c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\scipy\linalg\basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 1.6453525862009465e-24 / 1.1102230246251565e-16
  RuntimeWarning)
c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\scipy\linalg\basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 1.6627138698494546e-24 / 1.1102230246251565e-16
  RuntimeWarning)
c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\scipy\linalg\basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 6.235314068957496e-25 / 1.1102230246251565e-16
  RuntimeWarning)
c:\users\sakok\appdata\local\programs\python\python36-32

array([  5.66298835e-01,  -1.06377331e+01,  -1.62140065e-02,
        -1.53212238e-01,   1.08281857e-01,   1.69322025e-01,
        -6.36519190e+06,  -6.49858557e-01,   1.55843642e-01,
         3.85700014e-01])

In [130]:
# Small number of parameters.
lass = linear_model.Lasso(alpha=.7)
lassfit = lass.fit(X_train, y_train)
print('R² for the model with few features:')
print(lass.score(X_test, y_test))
origparams = np.append(lassfit.coef_, lassfit.intercept_)
print('\nParameter estimates for the model with few features:')
print(origparams)

R² for the model with few features:
0.204137761048

Parameter estimates for the model with few features:
[  1.41555975e-06  -4.64978097e-13  -0.00000000e+00  -0.00000000e+00
  -0.00000000e+00  -1.42615424e-04   2.05185165e-04   6.45321135e-02]


c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [ ]:
#In this case, using vanilla logistic regression yielded the best results. The penalty of L1 was used that allowed the model's 
#accuracy to be much better than L2. 